<a href="https://colab.research.google.com/github/milad-s5/DL_HW3_RNN/blob/main/1%20Sentiment140%20NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
cd 'drive/MyDrive/Colab Notebooks/DL/HW3/'

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/DL/HW3/'
/content/drive/MyDrive/Colab Notebooks/DL/HW3


In [42]:
! ls -all -h

total 6.5G
-rw------- 1 root root 5.5K Jan  1 09:38 1.ipynb
drwx------ 2 root root 4.0K Jan  1 09:08 data
-rw------- 1 root root 4.7G Oct 24  2015 glove.42B.300d.txt
-rw------- 1 root root 1.8G Oct 25  2015 glove.42B.300d.zip


In [21]:
# Make data directory if it doesn't exist
!mkdir -p data
!wget -nc http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip -P data
!unzip -n -d data "data/trainingandtestdata.zip"

URL transformed to HTTPS due to an HSTS policy
File ‘data/trainingandtestdata.zip’ already there; not retrieving.

Archive:  data/trainingandtestdata.zip


In [37]:
# Download and unzip glove.42B.300d file if it doesn't exist
!wget -nc http://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip -n "glove.42B.300d.zip"

File ‘glove.42B.300d.zip’ already there; not retrieving.

Archive:  glove.42B.300d.zip


In [38]:
! ls -all -h

total 6.5G
drwx------ 2 root root 4.0K Jan  1 09:08 data
-rw------- 1 root root 4.7G Oct 24  2015 glove.42B.300d.txt
-rw------- 1 root root 1.8G Oct 25  2015 glove.42B.300d.zip
-rw------- 1 root root 3.2K Jan  1 08:04 Untitled0.ipynb


In [39]:
! ls ./data -all -h

total 306M
-rw------- 1 root root  73K Mar  4  2010 testdata.manual.2009.06.14.csv
-rw------- 1 root root 228M Mar  4  2010 training.1600000.processed.noemoticon.csv
-rw------- 1 root root  78M Apr 22  2012 trainingandtestdata.zip


# Preprocessing

In [63]:
import pandas as pd

traindf = pd.read_csv("data/training.1600000.processed.noemoticon.csv",
                names=['polarity', 'id', 'date', 'query', 'user', 'text'],
                encoding='latin-1')
traindf.head()

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Update polarity
`polarity` column:
`0` : negative, `4` : positive

Change that to `0` and `1`

In [64]:
traindf.polarity.value_counts()

4    800000
0    800000
Name: polarity, dtype: int64

In [65]:
traindf.polarity = traindf.polarity.replace({0: 0, 4: 1})
traindf.polarity.value_counts()

1    800000
0    800000
Name: polarity, dtype: int64

## Remove unneeded columns

We don't need all those columns! Let's get rid of the ones that won't affect the sentiment.

In [66]:
traindf = traindf.drop(columns=['id', 'date', 'query', 'user'])

In [69]:
traindf

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


## Sample

To make the filesize a little smaller and pandas a little happier, let's knock this down to 500,000 tweets.